# Análise Descritiva dos Dados 

## Base: Crime Data in Brazil

### fonte: https://www.kaggle.com/inquisitivecrow/crime-data-in-brazil

### Para começar a 'add' estou importando as bibliotecas Pandas para fazer operações de sql e Seaborn par plotar gráficos.

In [1]:
import pandas as pd
import seaborn as sns

%matplotlib inline
%config IPCompleter.greedy=True

### Instalando e em seguida importando o psycopg2 que usarei para criar a conexão com o banco de dados

In [2]:
!pip install psycopg2

In [3]:
import psycopg2

In [5]:
conn = psycopg2.connect(host='localhost',database = 'Criminalidade',user='postgres',password='123456')

### Primeira análise rápida usando pg_stats

#### Foram escolhidos os campos que representam o nome do atributo, fração de nulos existentes em cada atributo, número de valores distintos, valores mais comuns e frequencia dos valores mais comuns

In [6]:
resultado = pd.read_sql_query("""select attname,null_frac,n_distinct,most_common_vals,most_common_freqs from pg_stats where schemaname = 'public' """,conn)

In [7]:
resultado

,attname,null_frac,n_distinct,most_common_vals,most_common_freqs
0,ano_bo,0.0,5.0,"{2016,2014,2013,2015,2017}","[0.3243, 0.236433, 0.2078, 0.203367, 0.0281]"
1,num_bo,0.0,25419.0,"{428,8,2006,2687,1,396,476,638,1669,2432,2605,...","[0.000466667, 0.000366667, 0.000366667, 0.0003..."
2,nome_departamento_circ,0.0,1.0,"{""DECAP ""}",[1.0]
3,nome_seccional_circ,0.0,8.0,"{""DEL.SEC.3Âº OESTE "",""...","[0.172633, 0.156567, 0.141267, 0.127267, 0.113..."
4,id_delegacia,0.0,347.0,"{900020,10211,10216,20249,10247,10326,10226,20...","[0.230333, 0.0192667, 0.0185667, 0.0184, 0.016..."
5,nome_departamento,0.0,17.0,"{""DECAP "",""D...","[0.7335, 0.230333, 0.0202667]"
6,nome_seccional,0.0,61.0,"{""DELEGACIA ELETRONICA "",""D...","[0.230333, 0.1174, 0.1114, 0.100167, 0.0885667..."
7,nome_delegacia,0.0,347.0,"{""DELEGACIA ELETRONICA "",""11Âº D.P. S...","[0.230333, 0.0192667, 0.0185667, 0.0184, 0.016..."
8,cidade,0.0,17.0,"{""S.PAULO ""}",[0.998133]
9,nome_delegacia_circ,0.0,94.0,"{""01Âº D.P. SE "",""03Âº D.P. ...","[0.0306667, 0.0230667, 0.0199667, 0.0193667, 0..."


####  Ao olhar o campo null_frac, aparentemente vemos uma base sem nenhum valor null, exceto no atributo logradouro onde tem baixíssimo indice de nulos(0.0033% da base), porém olhando com uma maior atenção encontraremos alguns problemas, se observar o atributo datahora_ocorrencia_bo nos campos most_common_vals e most_common_freq, verá que o valor mais comum não é null e sim uma String 'NULL' e além disso a frequencia é 1, ou seja, esse atributo é completamente inválido e deverá ser removido do banco, pois não agrega nenhum valor ás nossas análises.


### Dropando a coluna datahora_ocorrencia_bo

In [8]:
cur = conn.cursor()

In [9]:
cur.execute("""alter table crime drop column datahora_comunicacao_bo """)

In [10]:
pd.read_sql_query("""select attname,null_frac,n_distinct,most_common_vals,most_common_freqs from pg_stats where schemaname = 'public' """,conn)

,attname,null_frac,n_distinct,most_common_vals,most_common_freqs
0,ano_bo,0.0,5.0,"{2016,2014,2013,2015,2017}","[0.3243, 0.236433, 0.2078, 0.203367, 0.0281]"
1,num_bo,0.0,25419.0,"{428,8,2006,2687,1,396,476,638,1669,2432,2605,...","[0.000466667, 0.000366667, 0.000366667, 0.0003..."
2,nome_departamento_circ,0.0,1.0,"{""DECAP ""}",[1.0]
3,nome_seccional_circ,0.0,8.0,"{""DEL.SEC.3Âº OESTE "",""...","[0.172633, 0.156567, 0.141267, 0.127267, 0.113..."
4,id_delegacia,0.0,347.0,"{900020,10211,10216,20249,10247,10326,10226,20...","[0.230333, 0.0192667, 0.0185667, 0.0184, 0.016..."
5,nome_departamento,0.0,17.0,"{""DECAP "",""D...","[0.7335, 0.230333, 0.0202667]"
6,nome_seccional,0.0,61.0,"{""DELEGACIA ELETRONICA "",""D...","[0.230333, 0.1174, 0.1114, 0.100167, 0.0885667..."
7,nome_delegacia,0.0,347.0,"{""DELEGACIA ELETRONICA "",""11Âº D.P. S...","[0.230333, 0.0192667, 0.0185667, 0.0184, 0.016..."
8,cidade,0.0,17.0,"{""S.PAULO ""}",[0.998133]
9,nome_delegacia_circ,0.0,94.0,"{""01Âº D.P. SE "",""03Âº D.P. ...","[0.0306667, 0.0230667, 0.0199667, 0.0193667, 0..."


#### Em um segundo olhar, logo vemos que mais atributos tem a String 'NULL' como uns dos valores mais comuns, então agora investigaremos alguns desses atributos de forma um pouco mais minuciosa, começando pelo último atributo cor_cutis.

In [11]:
resultado3 = pd.read_sql_query("""select cor_cutis,count(*) from crime group by cor_cutis order by count(*) desc""",conn)

In [12]:
resultado3

,cor_cutis,count
0,Branca,794943
1,NULL,454486
2,Parda,267873
3,Preta,49322
4,Outros,20692
5,Amarela,14081
6,Vermelha,482
7,25,9
8,24,8
9,34,8


#### Com esse teste rapidamente percebemos 3 coisas, primeira de aproximadamente 1.602.000 registros, 2 em cor_cutis são realmente nulos, segunda os campos verdadeiramente nulos estão com a String 'NULL' e somam cerca de 400 mil registros, terceira apareceram vários números, logo, não são cores da pele. Vamos tentar descobrir a causa disso, vamos seleconar os dados onde cor_cutis é igual aos números.

In [13]:
resultado4 = pd.read_sql_query("""select * from crime where not (cor_cutis like '%Branca%') and not (cor_cutis like '%Parda%') and cor_cutis not like '%Amarela%' and cor_cutis not like '%Vermelha%' and cor_cutis not like '%Preta%'  and cor_cutis not like '%Outros%' and cor_cutis is not null and cor_cutis not like '%NULL%'

""",conn)

In [14]:
resultado4

,id_delegacia,nome_departamento,nome_seccional,nome_delegacia,cidade,ano_bo,num_bo,nome_departamento_circ,nome_seccional_circ,nome_delegacia_circ,...,descr_subtipolocal,logradouro,numero_logradouro,latitude,longitude,descr_tipo_pessoa,flag_vitima_fatal,sexo_pessoa,idade_pessoa,cor_cutis
0,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06Âº D.P. METROPOLITANO,S.PAULO,2013,302,DECAP,DEL.SEC.1Âº CENTRO,01Âº D.P. SE,...,Metrov. e ferroviÃ¡rio metrop.-Desembarque ...,ENTRE BRAS/SÃ‰,DESEMBARQUE,999,-23.54938522,-46.63325723,VÃ­tima,NULL,M,43
1,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06Âº D.P. METROPOLITANO,S.PAULO,2015,900016,DECAP,DEL.SEC.5Âº LESTE,30Âº D.P. TATUAPE,...,Metrov. e ferroviÃ¡rio metrop.-outros ...,ESTAÃ‡ÃƒO TATUAPÃ‰,MEZANINO AREA LIVRE,99,NULL,NULL,VÃ­tima,NULL,M,55
2,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06Âº D.P. METROPOLITANO,S.PAULO,2015,900016,DECAP,DEL.SEC.5Âº LESTE,30Âº D.P. TATUAPE,...,Metrov. e ferroviÃ¡rio metrop.-outros ...,ESTAÃ‡ÃƒO TATUAPÃ‰,MEZANINO AREA LIVRE,99,NULL,NULL,VÃ­tima,NULL,F,25
3,10217,DECAP,DEL.SEC.2Âº SUL,17Âº D.P. DOUTOR ALDO GALIANO,S.PAULO,2015,3505,DECAP,DEL.SEC.2Âº SUL,17Âº D.P. DOUTOR ALDO GALIANO,...,Via pÃºblica ...,RUA ISAURA ABAD GIACOMELLI,,87,NULL,NULL,VÃ­tima,NULL,M,25
4,10217,DECAP,DEL.SEC.2Âº SUL,17Âº D.P. DOUTOR ALDO GALIANO,S.PAULO,2015,3505,DECAP,DEL.SEC.2Âº SUL,17Âº D.P. DOUTOR ALDO GALIANO,...,Via pÃºblica ...,RUA ISAURA ABAD GIACOMELLI,,87,NULL,NULL,VÃ­tima,NULL,M,24
5,10226,DECAP,DEL.SEC.2Âº SUL,26Âº D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2Âº SUL,95Âº D.P. HELIÃ“POLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,VÃ­tima,NULL,F,51
6,10226,DECAP,DEL.SEC.2Âº SUL,26Âº D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2Âº SUL,95Âº D.P. HELIÃ“POLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,VÃ­tima,NULL,M,47
7,10226,DECAP,DEL.SEC.2Âº SUL,26Âº D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2Âº SUL,95Âº D.P. HELIÃ“POLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,VÃ­tima,NULL,F,79
8,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06Âº D.P. METROPOLITANO,S.PAULO,2013,2816,DECAP,DEL.SEC.1Âº CENTRO,06Âº D.P. CAMBUCI,...,Metrov. e ferroviÃ¡rio metrop.-vagÃ£o ...,ENTRE ESTAÂ§ÃƒO S.JOAQUIM,LIBERDADE,999,-23.57806782,-46.6405597,VÃ­tima,NULL,F,30
9,10248,DECAP,DEL.SEC.6Âº SANTO AMARO,48Âº D.P. CIDADE DUTRA,S.PAULO,2013,1372,DECAP,DEL.SEC.6Âº SANTO AMARO,101Âº D.P. JDIM IMBUIAS,...,Serralheria-Acesso,/Escada/Elevador/Passarela ...,R DR OSCAR ANDRADE LEMOS,910,-23.73392446,-46.69437827,VÃ­tima,NULL,M,56


#### Perceba que a linha por algum motivo deu um SHIFT na hora do ETL e corrompeu esses dados, ou seja, a idade está em cor_cutis, como achamos esse resultado no último atributo, todas as linhas afetadas por esse shift estão presentes, logo apagaremos todas essas linhas da nossa base de dados, juntamente com as linhas onde 'cor_cutis' = "NULL" e onde os dados são verdadeiramente nulos.

In [15]:
cur = conn.cursor()

In [16]:
cur.execute("""delete from crime where not (cor_cutis like '%Branca%') and not (cor_cutis like '%Parda%') and cor_cutis not like '%Amarela%' and cor_cutis not like '%Vermelha%' and cor_cutis not like '%Preta%'  and cor_cutis not like '%Outros%'""")

In [18]:
resultado5 = pd.read_sql_query("""select cor_cutis,count(*) from crime group by cor_cutis order by count(*) desc""",conn)

In [19]:
resultado5

,cor_cutis,count
0,Branca,794943
1,Parda,267873
2,Preta,49322
3,Outros,20692
4,Amarela,14081
5,Vermelha,482
6,None,2


#### Agora vamos analisar outras colunas que requerem uma formatação especial, por exemplo, a coluna hora_ocorrencia_bo que registra a hora da ocorrencia de um bo e está em um formato txt. 

In [20]:
resultado6 = pd.read_sql_query("""select hora_ocorrencia_bo,count(*) from crime group by hora_ocorrencia_bo order by count(*) desc""",conn)

In [21]:
resultado6

,hora_ocorrencia_bo,count
0,NULL,129003
1,20:00,23602
2,21:00,23248
3,22:00,22016
4,19:00,21124
5,14:00,18842
6,20:30,18724
7,23:00,18551
8,15:00,18289
9,12:00,17930


#### Ao final do resultado6, observamos que aparecem alguns dados em formato impróprio para a manipulação no banco de dados, agora faremos uma verificação se são os unicos casos em que isso ocorre ou se existem outros casos com mais número de registros onde isso ocorre, ou seja, onde hora_ocorrencia_bo é diferente de String 'NULL' e diferente de formato 12:00.

In [56]:
resultado7 = pd.read_sql_query("""select hora_ocorrencia_bo from crime where hora_ocorrencia_bo not like '%:%' and hora_ocorrencia_bo not like 'NULL'""",conn)

In [57]:
resultado7

,hora_ocorrencia_bo
0,15H50
1,15H50
2,1910
3,23H00
4,23H00


#### Como existem poucos dados que foram inseridos com uma formatação inadequada, faremos 3 alterações para mudar os valores desse atributo para o formato correto

In [ ]:
cur = conn.cursor()

In [24]:
cur.execute("""update crime set hora_ocorrencia_bo = '15:50' where hora_ocorrencia_bo = '15H50' """)

In [29]:
cur.execute("""update crime set hora_ocorrencia_bo = '23:00' where hora_ocorrencia_bo = '23H00' """)

In [28]:
cur.execute("""update crime set hora_ocorrencia_bo = '19:10' where hora_ocorrencia_bo = '1910' """)

In [30]:
resultado8 = pd.read_sql_query("""select hora_ocorrencia_bo from crime where hora_ocorrencia_bo not like '%:%' and hora_ocorrencia_bo not like 'NULL'""",conn)

In [55]:
resultado8

,hora_ocorrencia_bo
0,06:20:00
1,06:10:00
2,05:30:00
3,12:05:00
4,05:50:00
5,23:20:00
6,06:45:00
7,14:30:00
8,20:00:00
9,05:30:00


#### Agora vamos converter o horario para o formato certo

In [33]:
resultado9 = pd.read_sql_query("""select hora_ocorrencia_bo from crime where hora_ocorrencia_bo like '%:%' """,conn)

In [34]:
resultado9

,hora_ocorrencia_bo
0,06:20
1,06:10
2,05:30
3,12:05
4,05:50
5,23:20
6,06:45
7,14:30
8,20:00
9,05:30


#### Formatando o horario

In [36]:
resultado8['hora_ocorrencia_bo'] = pd.to_datetime(resultado9['hora_ocorrencia_bo'])

In [37]:
resultado8['hora_ocorrencia_bo'] = pd.to_datetime(resultado8['hora_ocorrencia_bo'], format= '%H:%M').dt.time

In [39]:
resultado8

,hora_ocorrencia_bo
0,06:20:00
1,06:10:00
2,05:30:00
3,12:05:00
4,05:50:00
5,23:20:00
6,06:45:00
7,14:30:00
8,20:00:00
9,05:30:00


### Aproveitando a onda dos horarios, vamos fazer um estudo de qual horario aparece mais.

In [53]:
resultado10 = pd.read_sql_query("""select hora_ocorrencia_bo, count(*) from crime group by hora_ocorrencia_bo order by count(*) """,conn)


In [54]:
resultado10

,hora_ocorrencia_bo,count
0,1910,1
1,23H00,2
2,15H50,2
3,05:09,16
4,04:41,18
5,04:21,18
6,04:02,20
7,04:22,21
8,05:14,22
9,04:33,22


In [52]:
sns.pairplot(resultado10)

IndexError: index -1 is out of bounds for axis 0 with size 0

<Figure size 0x0 with 0 Axes>